In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from pathlib import Path


In [18]:
data = Path('data/vader_emolex.csv')
df = pd.read_csv(data)
bins = [0, 2.5, 3.5, 5]
labels = ['Negative', 'Neutral', 'Positive']
df['bins'] = pd.cut(df['stars'], bins=bins, labels=labels)
n = 100000
subset = df.groupby('bins').head(n)

In [19]:
reviews = df[['text', 'stars']]

In [20]:
reviews['sentiment'] = reviews['stars'].apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(reviews['text'], reviews['sentiment'], test_size=0.2)

# Create a CountVectorizer to transform the text data into numerical features
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Train a logistic regression model on the training data
model = LogisticRegression()
model.fit(X_train_vect, y_train)

# Evaluate the model on the test data
score = model.score(X_test_vect, y_test)
print(f'Test accuracy: {score:.2f}')

/var/folders/mm/358j6d652pl4dwx_trrxq3lh0000gn/T/ipykernel_45732/3875057155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews['sentiment'] = reviews['stars'].apply(lambda x: 0 if x <= 2 else 1 if x == 3 else 2)
/Users/reinier/miniconda3/envs/python31/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_res

Test accuracy: 0.85


In [21]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_vect)
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))

              precision    recall  f1-score   support

    negative       0.83      0.86      0.84     44766
     neutral       0.54      0.29      0.38     22232
    positive       0.89      0.96      0.92    110725

    accuracy                           0.85    177723
   macro avg       0.75      0.70      0.72    177723
weighted avg       0.83      0.85      0.83    177723



In [22]:
testing_report = classification_report(y_test, testing_predictions)
print(testing_report)

NameError: name 'testing_predictions' is not defined

In [30]:
from joblib import dump
dump(model, 'model_LR.joblib')
dump(vectorizer, 'vectorizer_LR.joblib')

['vectorizer_LR.joblib']

In [29]:
sample_text = "The food was good and the service was fast."
sample_text_vect = vectorizer.transform([sample_text])
sentiment_pred = model.predict(sample_text_vect)[0]
sentiment_label = ['negative', 'neutral', 'positive'][sentiment_pred]
print(f'Sentiment prediction for sample text: {sentiment_label}')

Sentiment prediction for sample text: positive
